In [14]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [15]:
dataset = 'RM2009'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/pr9003773_si_002.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for RM2009 ...
Raw data loaded.


,Protein,Protein Names,Gene Names,Protein Descriptions,Uniprot,ENSEMBL,Mascot Score,m/z,Charge,Mass Error [ppm],...,Phospho_STY (STY) Probabilities,Sequence Window (the phosph. residue is centered in the middle of the sequence),Ratio M/L (M:15 min timepoint;L:0 min. timepoint),Ratio M/L Normalized by Corresponding Protein Level (M:15 min timepoint;L:0 min. timepoint),Ratio H/L (H:60 min timepoint;L:0 min. timepoint),Ratio H/L Normalized by Corresponding Protein Level (H:60 min timepoint;L:0 min. timepoint),Best Localization Raw File,Best Localization Scan Number,Best Identification Raw File,Best Identification Scan Number
0,IPI:IPI00012268,26S proteasome non-ATPase regulatory subunit 2...,PSMD2;TRAP2,26S proteasome non-ATPase regulatory subunit 2...,Q13200;Q53XQ4;Q59EG8;Q9NSM5,ENSG00000175166,33.09,847.73755,3,-0.022477,...,DKAPVQPQQS(1)PAAAPGGTDEKPSGK,PVQPQQSPAAAPG,0.80374,0.79208,0.71153,0.69237,T000842rl,4952,T000842rl,4952
1,IPI:IPI00221089,40S ribosomal protein S13,RPS13,40S ribosomal protein S13,P62277;B2R549,ENSG00000110700,35.49,759.85800,2,-0.042655,...,LT(0.989)S(0.011)DDVKEQIYK,PTWLKLTSDDVKE,0.62054,0.75575,0.25574,0.29479,T000880rl,5368,T000880rl,5368
2,IPI:IPI00029750,40S ribosomal protein S24;40S ribosomal protei...,RPS24;RPS24,Isoform 1 of 40S ribosomal protein S24;15 kDa ...,P62847;Q5T0P7;Q7Z3D1;Q5T0P8,ENSG00000138326;ENSG00000138326,13.58,544.22811,2,0.141700,...,MNDT(0.981)VT(0.019)IR,___MNDTVTIRTR,0.66005,0.86443,0.53412,0.59935,T000880rl,4111,T000880rl,4111
3,IPI:IPI00179330,"40S ribosomal protein S27a;cDNA FLJ75516, high...",RPS27A;UBA80;UBCEP1;UBC;UBB;hCG_1998947;UBC;UB...,ubiquitin and ribosomal protein S27a precursor...,P62979;B2RDW1;Q5RKT7;Q8WYN9;A8K674;Q59EM9;Q66K...,ENSG00000143947;ENSG00000150991;ENSG0000015099...,23.82,581.26301,2,0.032862,...,TLS(1)DYNIQK,EDGRTLSDYNIQK,0.80698,1.02920,1.32550,1.35260,T000874rl,5085,T000874rl,5085
4,IPI:IPI00011253,40S ribosomal protein S3,RPS3;OK/SW-cl.26,40S ribosomal protein S3;12 kDa protein,P23396;B2R7N5;Q53G83;Q6IPX9;Q9NQS8,ENSG00000149273,15.28,775.87111,2,-0.888070,...,DEILPT(0.919)T(0.08)PIS(0.001)EQK,KDEILPTTPISEQ,0.75476,0.93576,1.06270,1.06950,T000868rl,6177,T000599rl,5970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,25.26,958.45112,2,0.000847,...,EQQSWVIS(0.998)PPQPS(0.002)PQK,QQSWVISPPQPSP,0.86741,1.04950,3.95050,1.04600,T000838rl,8602,T000838rl,8602
1936,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,45.71,687.80049,2,-1.479500,...,S(0.803)ES(0.197)VEGFLSPSR,GKRTPRSESVEGF,0.87669,1.06070,3.49900,0.92646,T000837rl,9025,T000838rl,8929
1937,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,48.42,593.94728,3,0.041962,...,S(0.006)IS(0.994)NEGLTLNNSHVSK,GITRSISNEGLTL,0.87812,1.06250,3.79320,1.00440,T000458rl,5928,T000458rl,5920
1938,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,45.71,687.80049,2,-0.652220,...,SES(1)VEGFLSPSR,RTPRSESVEGFLS,0.89660,1.08480,3.30590,0.87535,T000838rl,8929,T000838rl,8929


In [16]:
# filter data to keep only those with localization probability >= 0.85
data = data[data['Phosphosite Localization Probability (within the phosphopeptide)'] >= 0.85] 
# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino Acid'].str.contains(';', na=False)]
data = data[~data['Position'].astype(str).str.contains(';', na=False)]
data = data[~data['Gene Names'].str.contains(';', na=False)]
data

,Protein,Protein Names,Gene Names,Protein Descriptions,Uniprot,ENSEMBL,Mascot Score,m/z,Charge,Mass Error [ppm],...,Phospho_STY (STY) Probabilities,Sequence Window (the phosph. residue is centered in the middle of the sequence),Ratio M/L (M:15 min timepoint;L:0 min. timepoint),Ratio M/L Normalized by Corresponding Protein Level (M:15 min timepoint;L:0 min. timepoint),Ratio H/L (H:60 min timepoint;L:0 min. timepoint),Ratio H/L Normalized by Corresponding Protein Level (H:60 min timepoint;L:0 min. timepoint),Best Localization Raw File,Best Localization Scan Number,Best Identification Raw File,Best Identification Scan Number
1,IPI:IPI00221089,40S ribosomal protein S13,RPS13,40S ribosomal protein S13,P62277;B2R549,ENSG00000110700,35.49,759.85800,2,-0.042655,...,LT(0.989)S(0.011)DDVKEQIYK,PTWLKLTSDDVKE,0.62054,0.75575,0.25574,0.29479,T000880rl,5368,T000880rl,5368
7,IPI:IPI00008433,40S ribosomal protein S5,RPS5,40S ribosomal protein S5,P46782;B2R4T2;Q53G25,ENSG00000083845,17.03,532.76044,2,0.252960,...,QAVDVS(1)PLR,RQAVDVSPLRRVN,0.63455,0.71371,0.54996,0.59879,T000874rl,4821,T000874rl,4821
12,IPI:IPI00024933,60S ribosomal protein L12;Uncharacterized prot...,RPL12,Isoform 1 of 60S ribosomal protein L12;Unchara...,P30050;Q59FI9;A8MTE8;A6NKJ6,ENSG00000197958;ENSG00000172746;ENSG00000204194,17.35,481.25954,2,-0.471220,...,IGPLGLS(1)PK,IGPLGLSPKKVGD,0.91031,1.20270,1.80380,1.86080,T000600rl,6677,T000880rl,7325
15,IPI:IPI00215719,60S ribosomal protein L18,RPL18,60S ribosomal protein L18;similar to hCG201418...,Q07020;Q0QEW2,ENSG00000063177,34.89,598.77831,2,-0.053975,...,GCGTVLLS(1)GPR,CGTVLLSGPRKGR,0.73756,0.85472,0.30227,0.31485,T000874rl,5521,T000874rl,5521
16,IPI:IPI00025329,60S ribosomal protein L19,RPL19,60S ribosomal protein L19;similar to ribosomal...,P84098;B2R4K2;Q53G49;Q8IWR8,ENSG00000108298,19.11,413.21514,2,0.343230,...,LASS(1)VLR,QKRLASSVLRCGK,0.78707,0.95436,0.55509,0.60576,T000873rl,4326,T000873rl,4326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,34.25,749.82469,2,-0.611080,...,LNQSS(1)PDNVTDTK,IKLNQSSPDNVTD,0.86734,1.04940,3.99610,1.05810,T000841rl,3263,T000838rl,3276
1935,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,25.26,958.45112,2,0.000847,...,EQQSWVIS(0.998)PPQPS(0.002)PQK,QQSWVISPPQPSP,0.86741,1.04950,3.95050,1.04600,T000838rl,8602,T000838rl,8602
1937,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,48.42,593.94728,3,0.041962,...,S(0.006)IS(0.994)NEGLTLNNSHVSK,GITRSISNEGLTL,0.87812,1.06250,3.79320,1.00440,T000458rl,5928,T000458rl,5920
1938,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,45.71,687.80049,2,-0.652220,...,SES(1)VEGFLSPSR,RTPRSESVEGFLS,0.89660,1.08480,3.30590,0.87535,T000838rl,8929,T000838rl,8929


In [17]:
data['Sequence Window (the phosph. residue is centered in the middle of the sequence)'] = data['Sequence Window (the phosph. residue is centered in the middle of the sequence)'].str.replace('_', '')
data

,Protein,Protein Names,Gene Names,Protein Descriptions,Uniprot,ENSEMBL,Mascot Score,m/z,Charge,Mass Error [ppm],...,Phospho_STY (STY) Probabilities,Sequence Window (the phosph. residue is centered in the middle of the sequence),Ratio M/L (M:15 min timepoint;L:0 min. timepoint),Ratio M/L Normalized by Corresponding Protein Level (M:15 min timepoint;L:0 min. timepoint),Ratio H/L (H:60 min timepoint;L:0 min. timepoint),Ratio H/L Normalized by Corresponding Protein Level (H:60 min timepoint;L:0 min. timepoint),Best Localization Raw File,Best Localization Scan Number,Best Identification Raw File,Best Identification Scan Number
1,IPI:IPI00221089,40S ribosomal protein S13,RPS13,40S ribosomal protein S13,P62277;B2R549,ENSG00000110700,35.49,759.85800,2,-0.042655,...,LT(0.989)S(0.011)DDVKEQIYK,PTWLKLTSDDVKE,0.62054,0.75575,0.25574,0.29479,T000880rl,5368,T000880rl,5368
7,IPI:IPI00008433,40S ribosomal protein S5,RPS5,40S ribosomal protein S5,P46782;B2R4T2;Q53G25,ENSG00000083845,17.03,532.76044,2,0.252960,...,QAVDVS(1)PLR,RQAVDVSPLRRVN,0.63455,0.71371,0.54996,0.59879,T000874rl,4821,T000874rl,4821
12,IPI:IPI00024933,60S ribosomal protein L12;Uncharacterized prot...,RPL12,Isoform 1 of 60S ribosomal protein L12;Unchara...,P30050;Q59FI9;A8MTE8;A6NKJ6,ENSG00000197958;ENSG00000172746;ENSG00000204194,17.35,481.25954,2,-0.471220,...,IGPLGLS(1)PK,IGPLGLSPKKVGD,0.91031,1.20270,1.80380,1.86080,T000600rl,6677,T000880rl,7325
15,IPI:IPI00215719,60S ribosomal protein L18,RPL18,60S ribosomal protein L18;similar to hCG201418...,Q07020;Q0QEW2,ENSG00000063177,34.89,598.77831,2,-0.053975,...,GCGTVLLS(1)GPR,CGTVLLSGPRKGR,0.73756,0.85472,0.30227,0.31485,T000874rl,5521,T000874rl,5521
16,IPI:IPI00025329,60S ribosomal protein L19,RPL19,60S ribosomal protein L19;similar to ribosomal...,P84098;B2R4K2;Q53G49;Q8IWR8,ENSG00000108298,19.11,413.21514,2,0.343230,...,LASS(1)VLR,QKRLASSVLRCGK,0.78707,0.95436,0.55509,0.60576,T000873rl,4326,T000873rl,4326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,34.25,749.82469,2,-0.611080,...,LNQSS(1)PDNVTDTK,IKLNQSSPDNVTD,0.86734,1.04940,3.99610,1.05810,T000841rl,3263,T000838rl,3276
1935,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,25.26,958.45112,2,0.000847,...,EQQSWVIS(0.998)PPQPS(0.002)PQK,QQSWVISPPQPSP,0.86741,1.04950,3.95050,1.04600,T000838rl,8602,T000838rl,8602
1937,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,48.42,593.94728,3,0.041962,...,S(0.006)IS(0.994)NEGLTLNNSHVSK,GITRSISNEGLTL,0.87812,1.06250,3.79320,1.00440,T000458rl,5928,T000458rl,5920
1938,IPI:IPI00893859,NaN,NaN,calmodulin regulated spectrin-associated prote...,B1APG5;B3KTI4;Q08AD1-1;B3KX64;Q08AD1-2,NaN,45.71,687.80049,2,-0.652220,...,SES(1)VEGFLSPSR,RTPRSESVEGFLS,0.89660,1.08480,3.30590,0.87535,T000838rl,8929,T000838rl,8929


In [18]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 
    

In [19]:
data = match_seq_to_genename(data, 'Sequence Window (the phosph. residue is centered in the middle of the sequence)')

PTWLKLTSDDVKE
Match found for sequence: ID: sp|P62277|RS13_HUMAN
Name: sp|P62277|RS13_HUMAN
Description: sp|P62277|RS13_HUMAN Small ribosomal subunit protein uS15 OS=Homo sapiens OX=9606 GN=RPS13 PE=1 SV=2
Number of features: 0
Seq('MGRMHAPGKGLSQSALPYRRSVPTWLKLTSDDVKEQIYKLAKKGLTPSQIGVIL...LVA')
Gene name match: <re.Match object; span=(82, 90), match='GN=RPS13'>
Match found: PTWLKLTSDDVKE -> RPS13
RQAVDVSPLRRVN
Match found for sequence: ID: sp|P46782|RS5_HUMAN
Name: sp|P46782|RS5_HUMAN
Description: sp|P46782|RS5_HUMAN Small ribosomal subunit protein uS7 OS=Homo sapiens OX=9606 GN=RPS5 PE=1 SV=4
Number of features: 0
Seq('MTEWETAAPAVAETPDIKLFGKWSTDDVQINDISLQDYIAVKEKYAKYLPHSAG...SNR')
Gene name match: <re.Match object; span=(80, 87), match='GN=RPS5'>
Match found: RQAVDVSPLRRVN -> RPS5
IGPLGLSPKKVGD
Match found for sequence: ID: sp|P30050|RL12_HUMAN
Name: sp|P30050|RL12_HUMAN
Description: sp|P30050|RL12_HUMAN Large ribosomal subunit protein uL11 OS=Homo sapiens OX=9606 GN=RPL12 PE=1 SV=1
N

In [20]:
data['Phosphosite'] = data['Amino Acid'].astype(str) + '(' + data['Position'].astype(str) + ')'

In [21]:
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Ratio' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,Ratio M/L (M:15 min timepoint;L:0 min. timepoint),Ratio M/L Normalized by Corresponding Protein Level (M:15 min timepoint;L:0 min. timepoint),Ratio H/L (H:60 min timepoint;L:0 min. timepoint),Ratio H/L Normalized by Corresponding Protein Level (H:60 min timepoint;L:0 min. timepoint)
1,T(29),RPS13,0.62054,0.75575,0.25574,0.29479
7,S(142),RPS5,0.63455,0.71371,0.54996,0.59879
12,S(38),RPL12,0.91031,1.20270,1.80380,1.86080
15,S(140),RPL18,0.73756,0.85472,0.30227,0.31485
16,S(13),RPL19,0.78707,0.95436,0.55509,0.60576
...,...,...,...,...,...,...
1934,S(588),CAMSAP2,0.86734,1.04940,3.99610,1.05810
1935,S(920),CAMSAP2,0.86741,1.04950,3.95050,1.04600
1937,S(453),CAMSAP2,0.87812,1.06250,3.79320,1.00440
1938,S(1302),CAMSAP2,0.89660,1.08480,3.30590,0.87535


In [22]:
Ratio_columns = [col for col in data.columns if 'Ratio' in col]
data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1845/881820786.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')


In [23]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.Dataframe>
    
    out:
    ====
    dataset: <pd.Dataframe> with log2 transformed values

    '''
    cols_to_transform = dataset[Ratio_columns]
    dataset[Ratio_columns] = cols_to_transform.apply(np.log2)
    print('Data has been log2 transformed.')
    return dataset


In [24]:
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}") # Print the DataFrame after log2 transformation
data

Data has been log2 transformed.
DataFrame after log2 transformation:
     Phosphosite GeneName  Ratio M/L (M:15 min timepoint;L:0 min. timepoint)  \
1          T(29)    RPS13                                          -0.688404   
7         S(142)     RPS5                                          -0.656194   
12         S(38)    RPL12                                          -0.135570   
15        S(140)    RPL18                                          -0.439168   
16         S(13)    RPL19                                          -0.345436   
...          ...      ...                                                ...   
1934      S(588)  CAMSAP2                                          -0.205330   
1935      S(920)  CAMSAP2                                          -0.205214   
1937      S(453)  CAMSAP2                                          -0.187510   
1938     S(1302)  CAMSAP2                                          -0.157464   
1939     S(1137)  CAMSAP2                          

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1845/1555182160.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[Ratio_columns] = cols_to_transform.apply(np.log2)


,Phosphosite,GeneName,Ratio M/L (M:15 min timepoint;L:0 min. timepoint),Ratio M/L Normalized by Corresponding Protein Level (M:15 min timepoint;L:0 min. timepoint),Ratio H/L (H:60 min timepoint;L:0 min. timepoint),Ratio H/L Normalized by Corresponding Protein Level (H:60 min timepoint;L:0 min. timepoint)
1,T(29),RPS13,-0.688404,-0.404019,-1.967250,-1.762241
7,S(142),RPS5,-0.656194,-0.486590,-0.862601,-0.739878
12,S(38),RPL12,-0.135570,0.266277,0.851039,0.895923
15,S(140),RPL18,-0.439168,-0.226476,-1.726090,-1.667263
16,S(13),RPL19,-0.345436,-0.067395,-0.849206,-0.723182
...,...,...,...,...,...,...
1934,S(588),CAMSAP2,-0.205330,0.069565,1.998593,0.081476
1935,S(920),CAMSAP2,-0.205214,0.069702,1.982035,0.064883
1937,S(453),CAMSAP2,-0.187510,0.087463,1.923415,0.006334
1938,S(1302),CAMSAP2,-0.157464,0.117429,1.725043,-0.192068


In [25]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1845/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [26]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data

In [27]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

There are no phosphosites with multiple measurements
      Ratio M/L (M:15 min timepoint;L:0 min. timepoint)  \
1                                             -0.688404   
7                                             -0.656194   
12                                            -0.135570   
15                                            -0.439168   
16                                            -0.345436   
...                                                 ...   
1934                                          -0.205330   
1935                                          -0.205214   
1937                                          -0.187510   
1938                                          -0.157464   
1939                                          -0.125211   

      Ratio M/L Normalized by Corresponding Protein Level (M:15 min timepoint;L:0 min. timepoint)  \
1                                             -0.404019                                             
7                                   

,phosphosite_ID,Ratio M/L (M:15 min timepoint;L:0 min. timepoint),Ratio M/L Normalized by Corresponding Protein Level (M:15 min timepoint;L:0 min. timepoint),Ratio H/L (H:60 min timepoint;L:0 min. timepoint),Ratio H/L Normalized by Corresponding Protein Level (H:60 min timepoint;L:0 min. timepoint)
1,RPS13_T(29),-0.688404,-0.404019,-1.967250,-1.762241
7,RPS5_S(142),-0.656194,-0.486590,-0.862601,-0.739878
12,RPL12_S(38),-0.135570,0.266277,0.851039,0.895923
15,RPL18_S(140),-0.439168,-0.226476,-1.726090,-1.667263
16,RPL19_S(13),-0.345436,-0.067395,-0.849206,-0.723182
...,...,...,...,...,...
1934,CAMSAP2_S(588),-0.205330,0.069565,1.998593,0.081476
1935,CAMSAP2_S(920),-0.205214,0.069702,1.982035,0.064883
1937,CAMSAP2_S(453),-0.187510,0.087463,1.923415,0.006334
1938,CAMSAP2_S(1302),-0.157464,0.117429,1.725043,-0.192068


In [28]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/RM2009.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

RM2009 has been saved to CSV successfully!        phosphosite_ID  Ratio M/L (M:15 min timepoint;L:0 min. timepoint)  \
1         RPS13_T(29)                                          -0.688404   
7         RPS5_S(142)                                          -0.656194   
12        RPL12_S(38)                                          -0.135570   
15       RPL18_S(140)                                          -0.439168   
16        RPL19_S(13)                                          -0.345436   
...               ...                                                ...   
1934   CAMSAP2_S(588)                                          -0.205330   
1935   CAMSAP2_S(920)                                          -0.205214   
1937   CAMSAP2_S(453)                                          -0.187510   
1938  CAMSAP2_S(1302)                                          -0.157464   
1939  CAMSAP2_S(1137)                                          -0.125211   

      Ratio M/L Normalized by Corresponding 